# Baseline LSTM model

## Notebook set-up

In [1]:
# Standard library imports
import os
import pickle
import logging

# PyPI imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_score, recall_score, roc_curve, precision_recall_curve

# Set GPU for TensorFlow
os.environ['CUDA_VISIBLE_DEVICES']='2'

# Suppress warning and info messages from TensorFlow
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # FATAL
logging.getLogger('tensorflow').setLevel(logging.FATAL)

# Internal imports
import perdrizet_helper_funcs

# Input data
data_file='../data/resampled_disaster_data_all.parquet'

# Incident feature to use
incident_feature='linear_log_incidents'

# Training details
include_synthetic_features=False
window=1
training_iterations=100
training_epochs=5
learning_rate=0.0001
l1_weight=0.0001
l2_weight=0.001

## 1. Data loading

In [ ]:
raw_data_df=pd.read_parquet(data_file)

# Grab the features we are interested in
data_df=raw_data_df[['month_cos', 'month_sin', 'incidents_binary', incident_feature]].copy()

# Transfer the index
data_df.set_index(raw_data_df.index)

# Save the index of the target column
target_column_index=data_df.columns.get_loc(incident_feature)
data_df.head()

# Save the index of the target column
target_column_index=data_df.columns.get_loc('incidents_binary')

# Get and show some summary statistics
total_disaster_months=len(data_df[data_df['incidents_binary'] != 0])
percent_disaster_months=(total_disaster_months/len(data_df)) * 100
print(f'Have {total_disaster_months}({percent_disaster_months:.1f}%) disaster months\n')

data_df.info()

In [ ]:
data_df.describe()

In [ ]:
# Draw a quick plot to check the distribution of disaster counts
plt.title('Incident distribution')
plt.hist(data_df[incident_feature], bins=30, color='black')
plt.xlabel('Incidents')
plt.ylabel('Count')
plt.yscale('log')
plt.savefig('./figures/3-1-baseline_LSTM_incident_distribution.jpg')
plt.show()

In [ ]:
data_df.head()

## 2. Train-test split

Take the most recent ~10% of the data for testing.

In [ ]:
# Get list of years
years=data_df.index.get_level_values('year').unique().tolist()
testing_years=len(years) // 10
print(f'Using most recent {testing_years} of {len(years)} years for test set')
print(f'Testing years: {years[-testing_years:]}')
print(f'Training years: {years[:-testing_years]}')

# Take last n years for testing data
testing_df=data_df.loc[years[-testing_years:]]

# Take the rest for training
training_df=data_df.loc[years[:-testing_years]]

# Check result
training_df.head()

## 3. Data formatting & splitting

### 3.1. Training and validation data

In [ ]:
# Run custom splitting function
training_features, training_labels, validation_features, validation_labels, states=perdrizet_helper_funcs.make_windowed_time_course(training_df, target_column_index, window)

# Check the result
print(f'State batches: {len(training_features)}')

for state, features, labels in zip(states, training_features, training_labels):
    print(f'{state}: features: {features.shape}, labels: {labels.shape}')

### 3.2. Testing data

In [ ]:
# Format test data for predictions
testing_features=[]
testing_labels=[]

for i in range(len(testing_df) - window - 1):

    testing_features.append(testing_df.iloc[i:i + window])
    testing_labels.append([testing_df.iloc[i + window + 1,target_column_index]])

# Check the result
print(f'Testing features shape: {np.array(testing_features).shape}')
print(f'Testing labels shape: {np.array(testing_labels).shape}')

## 4. Initial model training

### 4.1. Set-up and build model

In [ ]:
# Build the model
model=perdrizet_helper_funcs.build_lstm(
    training_features[0].shape[1],
    training_features[0].shape[2],
    learning_rate=learning_rate,
    l1_weight=l1_weight,
    l2_weight=l2_weight
)

# Print out the model structure
model.summary()

### 4.2. Naive model test set performance

In [10]:
# # Make the predictions
# predictions=[]

# for features in testing_features:
#     predictions.extend(model.predict(np.array([features]), verbose=0))

# predictions_df=pd.DataFrame.from_dict({'labels': np.array(testing_labels).flatten(), 'probabilities': np.array(predictions).flatten()})
# predictions_df.head()

# # Set threshold and call incidents
# threshold=0.5
# calls=np.where(predictions_df['probabilities'] > threshold, 1, 0)

# # Calculate precision and recall
# precision=precision_score(predictions_df['labels'], calls)
# recall=recall_score(testing_labels, calls)
# print(f'Precision: {precision:.3f}')
# print(f'Recall: {recall:.3f}\n')

# # Plot the confusion matrix
# cm=confusion_matrix(predictions_df['labels'], calls, normalize='true')
# cm_disp=ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=['no incident', 'incident'])
# _=cm_disp.plot()
# plt.title('Naive LSTM test set performance')
# plt.savefig('./figures/3.4.2-naive_baseline_LSTM_test_set_performance.jpg')
# plt.show()

### 4.3. Train the model

In [ ]:
%%time

# Get the class weights
class_weight=perdrizet_helper_funcs.get_class_weights(training_df)

# Manually train the model iteratively over the state time courses using each for a single batch training run
true_epoch=0
training_results=[]
metrics=['loss', 'precision', 'recall', 'val_loss', 'val_precision', 'val_recall']

for iteration in range(training_iterations):

    # Make an empty dict to store metric results for this iteration
    metric_data={}

    for metric in metrics:
        metric_data[metric]=[]

    # Loop over the state time courses
    for i in range(len(training_features)):

        # Train on this state
        batch_result, model=perdrizet_helper_funcs.train_lstm(
            model,
            training_features[i],
            training_labels[i],
            validation_features[i],
            validation_labels[i],
            class_weight,
            epochs=training_epochs,
            batch_size=training_features[i].shape[0]
        )

        # Count total training epochs
        true_epoch+=training_epochs

        # Collect the state-level training result
        training_results.append(batch_result)

        # Collect metrics for state-level training run
        for metric in metrics:
            metric_data[metric].extend(batch_result.history[metric])

    # Every 10th iteration, save results and print performance summary
    if iteration % 10 == 0:

        model.save(f'../data/model_checkpoints/baseline_LSTM/epoch_{true_epoch}.keras')

        with open(f'../data/training_results/baseline_LSTM.pkl', 'wb') as output_file:
            pickle.dump(training_results, output_file)

        print(f'Training iteration {iteration} mean: ', end='')

        for metric in metrics:
            print(f'{metric}={sum(metric_data[metric])/len(metric_data[metric]):.3f} ', end='')
        
        print()

        plot=perdrizet_helper_funcs.plot_single_training_run(
            'Baseline LSTM training curves',
            training_results,
            len(states),
            training_epochs
        )
        plot.savefig('./figures/3-4.3-baseline_LSTM_training_curves.jpg')
        plot.close()
        
# Draw the final plot after the last loop
plot.show()

## 5. Model evaluation

### 5.1. Make predictions on test set

In [ ]:
# Make the predictions
predictions=[]

for features in testing_features:
    predictions.extend(model.predict(np.array([features]), verbose=0))

predictions_df=pd.DataFrame.from_dict({'labels': np.array(testing_labels).flatten(), 'probabilities': np.array(predictions).flatten()})
predictions_df.head()

### 5.2. Distribution of predicted incident probabilities

In [ ]:
plt.title('Baseline features LSTM: predicted test set probabilities')
plt.hist(predictions_df['probabilities'][predictions_df['labels'] == 0], bins=30, density=True, alpha=0.5, label='No incident')
plt.hist(predictions_df['probabilities'][predictions_df['labels'] == 1], bins=30, density=True, alpha=0.5, label='Incident')
plt.xlabel('Predicted probability of incident')
plt.ylabel('Density')
plt.legend(loc='upper left')
plt.savefig('./figures/3-5.2-baseline_LSTM_predicted_probability_distributions.jpg')
plt.show()

### 5.3. Receiver-operator characteristic and precision-recall curves

In [ ]:
# Set-up a 1x2 figure
fig, axs=plt.subplots(1,2, figsize=(8,4))

# Add the main title
fig.suptitle('Baseline LSTM test set performance', size='large')

# Plot ROC curve
fp, tp, _ = roc_curve(predictions_df['labels'], predictions_df['probabilities'])
axs[0].set_title('ROC curve')
axs[0].plot(100*fp, 100*tp, color='black')
axs[0].set_xlabel('False positives [%]')
axs[0].set_ylabel('True positives [%]')
axs[0].grid(True)
axs[0].set_aspect('equal')

# Plot PR curve
precision, recall, _ = precision_recall_curve(predictions_df['labels'], predictions_df['probabilities'])
axs[1].set_title('Precision-recall curve')
axs[1].plot(precision, recall, color='black')
axs[1].set_xlabel('Precision')
axs[1].set_ylabel('Recall')
axs[1].grid(True)
axs[1].set_aspect('equal')

plt.savefig('./figures/3-5.3-baseline_LSTM_ROC_PR_curves.jpg')
plt.show()

### 5.4. Confusion matrix

In [ ]:
# Set threshold and call incidents
threshold=0.5
calls=np.where(predictions_df['probabilities'] > threshold, 1, 0)

# Calculate precision and recall
precision=precision_score(predictions_df['labels'], calls)
recall=recall_score(testing_labels, calls)
print(f'Precision: {precision:.3f}')
print(f'Recall: {recall:.3f}\n')

# Plot the confusion matrix
cm=confusion_matrix(predictions_df['labels'], calls, normalize='true')
cm_disp=ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=['no incident', 'incident'])
_=cm_disp.plot()
plt.title('Baseline LSTM test set performance')
plt.savefig('./figures/3-5.4-baseline_LSTM_confusion_matrix.jpg')
plt.show()